<a href="https://colab.research.google.com/github/PoriyaAbdollahi/TradingBot/blob/master/Tradingbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare Librarys  And dataset for training

In [ ]:
!pip install stable-baselines3 gym-anytrading gym==0.25.2 yahoo_fin

In [ ]:
# Gym Stuff
import gym
import gym_anytrading

# Stable base line rl stuff
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C  

# processing libs
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 

# data downloader
from yahoo_fin import stock_info as si
import datetime

In [ ]:
# use local data
# df = pd.read_csv('data.csv')

# download data from yahoo finance 
# Define the ticker symbol for MATIC-USD
ticker = "MATIC-USD"

# Calculate the start and end dates for the five-year period
end_date = datetime.datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.datetime.now() - datetime.timedelta(days=5*365)).strftime('%Y-%m-%d')

# Download the historical data using yahoo_fin
data = si.get_data(ticker, start_date, end_date)

# Convert the data to a DataFrame
df = pd.DataFrame(data)

# Rename the index column to "Date"
df = df.rename_axis("Date").reset_index()

# Display the DataFrame
print(df)


In [ ]:
df.head()

In [ ]:
del df["adjclose"]
del df["ticker"]
df = df.rename(columns={'open':'Open' , 'high':'High' , 'low':'Low' , 'close':'Close' ,'volume':'Volume'})

In [ ]:
df.dtypes

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.dtypes

In [ ]:
df.set_index('Date',inplace=True)
df.head()

# Environment setup

In [ ]:
env = gym.make('stocks-v0' , df=df , frame_bound=(10,1500) , window_size=10)

In [ ]:
env.signal_features

In [ ]:
state = env.reset()
while True:
  action = env.action_space.sample()
  n_state,reward , done , info = env.step(action)
  if done:
    print('info',info)
    break
plt.figure(figsize=(15,6))
plt.cla()
env.render_all()
plt.show()        

In [ ]:
env.reset()

# Training


In [ ]:
env_maker = lambda:gym.make('stocks-v0',df=df, frame_bound=(10,1500),window_size=10)
env= DummyVecEnv([env_maker])

In [ ]:
import os 
log_path = os.path.join('Training' , 'Logs')
model = A2C('MlpPolicy',env ,verbose=1 ,tensorboard_log=log_path)
model.learn(total_timesteps=100000)

In [ ]:
model.learn(total_timesteps=1000000)

In [ ]:

env = gym.make('stocks-v0' , df=df, frame_bound =( 130,160 ),window_size=10)
obs = env.reset()
while True:
  obs = obs[np.newaxis, ...]
  action , _states = model.predict(obs)
  obs ,rewards , done , info = env.step(action)
  if done:
    print('info',info)
    break

In [ ]:
plt.figure(figsize=(15,6))
plt.cla()
env.render_all()
plt.show()        

# Training using Indicatiors

In [ ]:
!pip install finta

In [ ]:
from gym_anytrading.envs import StocksEnv
from finta import TA

In [ ]:
df['SMA']  = TA.SMA(df,12)
df['RSI']  = TA.RSI(df)
df['OBV']  = TA.OBV(df)
df.fillna(0,inplace=True)

In [ ]:
df.head()

In [ ]:
def add_signals(env):
  start = env.frame_bound[0] - env.window_size
  end = env.frame_bound[1] 
  prices = env.df.loc[:,'Low'].to_numpy()[start:end]
  signal_features = env.df.loc[:,['Low' , 'Volume' , 'SMA', 'RSI' , 'OBV' ]].to_numpy()[start:end]
  return prices , signal_features


In [ ]:
class MyCustomEnv(StocksEnv):
  _process_data = add_signals

env2=  MyCustomEnv(df=df , window_size = 12 , frame_bound = (12,1500))

In [ ]:
env_maker = lambda:env2
env2= DummyVecEnv([env_maker])

In [ ]:
import os 
log_path = os.path.join('Training' , 'Logs')
model = A2C('MlpPolicy',env2 ,verbose=1 ,tensorboard_log=log_path)
model.learn(total_timesteps=1000000)

In [ ]:
env2 = MyCustomEnv( df=df, window_size=12,frame_bound=(1430,1460))
obs = env2.reset()
while True:
  obs = obs[np.newaxis, ...]
  action , _states = model.predict(obs)
  obs ,rewards , done , info = env2.step(action)
  if done:
    print('info',info)
    break

In [ ]:
plt.figure(figsize=(15,6))
plt.cla()
env2.render_all()
plt.show()    

# Save Model


In [ ]:
model.save("a2cModel")


# Load Pre Trained Model

In [ ]:
model = A2C.load('/content/drive/MyDrive/MyModels/a2cModel')

In [ ]:
import tensorflow as tf

# Replace 'path_to_saved_model' with the path to your SavedModel directory
loaded_model = tf.saved_model.load('A2CModel')
env = gym.make('stocks-v0' , df=df , frame_bound=(5,100) , window_size=5)
env.model = loaded_model